Loading data (MatExample)

In [1]:
import scipy.io
import numpy as np
import os
os.chdir('C:\\Users\\adam1brownell' + '\Documents\Dev101\MontiLab')

subj = "01" #Within Subject Models

#AB Data Load
abLoad = scipy.io.loadmat("sub-"+subj+"_task-analogy_ab-betas-ba10.mat")
abData    = abLoad["data"]      # actual data (numpy array)
abAnalogy = abLoad["trial_id"]  # actual analogy they saw
abArray   = abLoad["main_rel"]  # main relationship (1,2,3)
abRel     = [abArray[0,i][0]for i in range(288)] #I'm cheating and now there are 288 examples
abSub     = abLoad["sub_rel"]   # sub relationship (1-9)

#CD Data Load
cdLoad = scipy.io.loadmat("sub-"+subj+"_task-analogy_cd-betas-ba10.mat")
cdData    = cdLoad["data"]      # actual data (numpy array)
cdAnalogy = cdLoad["trial_id"]  # actual analogy they saw
cdArray   = cdLoad["main_rel"]  # main relationship (1,2,3)
cdRel     = [cdArray[0,i][0]for i in range(288)] #I'm cheating and now there are 288 examples
cdSub     = cdLoad["sub_rel"]   # sub relationship (1-9)


In [70]:
# Load Human Data
import pandas as pd

humanData = pd.read_csv('C:\\Users\\adam1brownell\\Documents\\GitHub\\task-fmri-utils\\humanratings.csv', header = 2)
humanData.columns = humanData.columns.str.replace(humanData.columns[0],'Analogy')

In [41]:
# Word2Vec can represent the way the brain stores analogous relations
# Ridge Regression on the Imported data

In [42]:
# Pipeline

# Input of a new dataset
# For each voxel:
    # Test correlation of each voxel output to the feature
    # Ridge regression with y being the voxel and x being input data

In [6]:
# Load Voxel Data
%matplotlib inline
projecttitle = 'Analogy'
import sys
sys.path.append(os.path.join("/Users", "adam1brownell", "Documents", "GitHub", "task-fmri-utils"))

import fmri_core as pa
import numpy as np
# Load Entire Grey Matter
imgFile= "C:\\Users\\adam1brownell\\Documents\\GitHub\\task-fmri-utils\\analogy001_grayMatter_betas_LSA.nii"
labFile=  "C:\\Users\\adam1brownell\\Documents\\GitHub\\task-fmri-utils\\sub-01_task-analogy_events-pymvpa.tsv"
mask=    "C:\\Users\\adam1brownell\\Documents\\GitHub\\task-fmri-utils\\aal-LOFC-bin_mask.nii"

maskedImg = pa.utils.maskImg(imgFile, mask)
labels = pa.utils.loadLabels(labFile, sep='\t', index_col=0)

#Apply Mask
maskr = pa.utils.loadImg(mask)
imgr = pa.utils.loadImg(imgFile)
maskd = np.array(maskr.get_data())
imgd = np.array(imgr.get_data())

Masking C:\Users\adam1brownell\Documents\GitHub\task-fmri-utils\analogy001_grayMatter_betas_LSA.nii
Loading label file from: C:\Users\adam1brownell\Documents\GitHub\task-fmri-utils\sub-01_task-analogy_events-pymvpa.tsv
Reading file from: C:\Users\adam1brownell\Documents\GitHub\task-fmri-utils\aal-LOFC-bin_mask.nii
Reading file from: C:\Users\adam1brownell\Documents\GitHub\task-fmri-utils\analogy001_grayMatter_betas_LSA.nii


In [7]:
labels.columns

Index(['ab', 'abmainrel', 'absubrel', 'abtag', 'cd', 'cdmainrel', 'cdsubrel',
       'cdtag', 'chunks', 'match', 'probe', 'probearr', 'probecorr',
       'proberesp', 'trialtag', 'trialtype'],
      dtype='object')

In [123]:
# analogOrder is a function that scrapes the AB analogy from each trial in
# labels and then adds the row that corresponds to that analogy

# It currently assumes that newData is a panda with a column entitled 'Analogy'
# Reminder that labels will have more rows that newData since they ran the experiment

def analogyOrder(newData,labels):
    import re

    analogyDF = pd.DataFrame() #Initialize empty DF

    for row in range(len(labels)): #For every trial in the testing data...
    
        # Pull analogy for analogy pair, which is saved 'word1:word2::word3:word4' for AB + CD trials
        analogy = re.split("::", labels.trialtag[row])[0]
        
        # Find appropriate analogy data from newData and append to new DF
        newAnalogy = newData.loc[newData['Analogy'] == analogy]
        analogyDF = pd.concat([analogyDF,newAnalogy])
        
    return analogyDF
        
        

In [10]:
# searlightCorr calculates correlation of newData to each voxel from fMRI voxels
# This is done by running Ridge regression on both the newData and 
# a dummy regressor, where x = data and y = voxel value per trial

# TODO: Build Flag for Voxels bc searchlight takes forever

# TODO: Run a test to see how it will perform!

def searchlightCorr(newData, voxelData, labels):
    from sklearn import preprocessing
    from sklearn.linear_model import Ridge
    from sklearn.dummy import DummyRegressor #It is a regressor because voxel values are continuous
    
    
    # new Data should be [n x m] matrix of prediction
    # voxel Data should be [n x z] matrix of voxel outputs
    
    #orderedData will be a DF that is in the correct order of analogy pairs, according to voxel labels
    orderedData = analogyOrder(newData,labels)
    
    RSSList = [] #Will hold RSS 
    CorrList = [] #Will hold difference between dummy and new data
    # Scale newData
    scaleData = preprocessing.scale(newData)
    
    for col in range(voxelData.shape[1]): # this spits out columns from voxel data
        vox = voxelData[:,i]              # Voxels are the columns
        ridge = Ridge()
        dummy = DummyRegressor()
        
        # Fit Ridge + dummy versus this vox
        ridge.fit(scaleData,vox)
        dummy.fit(scaleData,vox)
        
        # Predict
        ridgePred = ridge.predict(vox)
        dummyPred = dummy.predict(vox)
        
        # R**2 value
        ridgeRSS = sum((ridgePred-vox)**2)
        dummyRSS = sum((dummyPred-vox)**2)
        
        # Find difference and add to list
        RSSList.append(ridgeRSS)
        CorrList.append(ridgeRSS-dummyRSS)
        
    import matplotlib.pyplot as plt
    
    #Build X axis as long as number of voxels
    voxList = [i for i in range(voxelData.shape[1])]
    
    #Plot both ridge RSS and difference
    plt.plot(voxList,RSSList, 'b', label = 'RSS')
    plot.plot(voxList, CorrList, 'r', label = 'Performance vs. Dummy')
    plot.yscale([-1,1])
    plot.title("Correlation per Voxel")
    
    #Print Cumulative Success
    print("Data Performance: ",float(sum(RSSList))/len(RSSList) )
    print("Dummy Performance: ",float(sum(RSSList - CorrList))/len(RSSList) )
        
        
        
    